In [2]:
import numpy as np

In [3]:
import mrcnn.utils
import mrcnn.config
import mrcnn.model

/content/Mask-RCNN-TF2/mrcnn/model.py:2370: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if os.name is 'nt':


AttributeError: ignored